![](https://images.pexels.com/photos/1167160/pexels-photo-1167160.jpeg?auto=compress&cs=tinysrgb&dpr=2&h=750&w=1260)

# Euler Angles and Quaternions for Rigid Body Rotations

Kevin J. Walchko, Phd

1 Nov 2020

---

Unit quaternions provide a convenient mathematical notation for representing orientations and rotations of objects in three dimensions. Compared to Euler angles they are able to avoid the problem of gimbal lock and are more compact than rotation matrices.

This document will start off talking about Euler angles (because they are more
intuitive to the reader) and then move into quaternions. The goal of this document is:

- be brief, it is not meant to be a disertation
- be clear in the explanation of rotations
    - use references for support or for further detailed discussion

### Table of Contents

- Euler Angles and Rotations
- Quaternions
- Appendix A: Quaternion Symbolic Derivation
- Appendix B: Euler angle symbolic derivation
- Appendix C: Definitions

## References

- MIT 16333: [Lecture 3]()
- Simo Sarkka, [Notes on Quaternions](../quaternion.pdf)
- Sympy docs: [Quaternions](https://www.tutorialspoint.com/sympy/sympy_quaternion.htm)
- [Quaternion Integration](https://www.ashwinnarayan.com/post/how-to-integrate-quaternions/)
- Wikipedia: [Convert Between Quaternions and Euler Angles](https://en.wikipedia.org/wiki/Conversion_between_quaternions_and_Euler_angles)
- Wikipedia: [Euler Angle Definitions](https://en.wikipedia.org/wiki/Euler_angles#Conventions_2)
- Wikipedia: [Gimbal Lock](https://en.wikipedia.org/wiki/Gimbal_lock)
- Wikipedia: [Aircraft Axes](https://en.wikipedia.org/wiki/Aircraft_principal_axes)

In [1]:
# reload library
%load_ext autoreload
%autoreload 2

In [2]:
# for real mathematics
from squaternion import Quaternion
from math import pi
from math import sqrt, atan2, asin, acos
import numpy as np
from numpy.testing import assert_allclose
np.set_printoptions(precision=1)
np.set_printoptions(suppress=True)
from colorama import Fore

# 1.0 Euler Angles

## 1.1 Euler Angles and Rigid Body Rotations

The fundamental terms most  people understand in 3D space are: roll ($\phi$), pitch ($\theta$), yaw ($\psi$). The fundamental rotation matricies are:

$$
R_1(\phi) =
\left[ \begin{matrix}
1 & 0 & 0 \\
0 & c_{\phi} & s_{\phi} \\
0 & -s_{\phi} & c_{\phi} \end{matrix} \right] \\
R_2(\theta) =
\left[ \begin{matrix}
c_{\theta} & 0 & -s_{\theta} \\
0 & 1 & 0 \\
s_{\theta} & 0 & c_{\theta} \end{matrix} \right] \\
R_3(\psi) =
\left[ \begin{matrix}
c_{\psi} & s_{\psi} & 0  \\
-s_{\psi} & c_{\psi} & 0 \\
0 & 0 & 1 \end{matrix} \right]
$$

Unfortunately, there **isn't** consistent notation used in the 
engineering community, so becareful! I use $R_{zyx}(rot_z,rot_y,rot_x)$ and $R_{xyz}(rot_x,rot_y,rot_z)$, both axis and angles are in order of operation from left to right.

The standard aerospace rotation sequence is:

![](https://upload.wikimedia.org/wikipedia/commons/thumb/5/53/Taitbrianzyx.svg/400px-Taitbrianzyx.svg.png)

> Tait–Bryan angles. z-y′-x″ sequence (intrinsic rotations; N coincides with y’). The angle rotation sequence is ψ, θ, φ. Note that in this case ψ > 90° and θ is a negative angle.

*Warning*, the wikipedia diagram above uses X,Y,Z for body frame and x,y,z for fixed global frame ... I do the opposite. 

1. Body and global fixed frames are aligned
    - Typically the global fixed frame is the static inertial frame
    - Global fixed frame $\begin{bmatrix}X & Y & Z\end{bmatrix}^T$
    - Body frame $\begin{bmatrix}x & y & z\end{bmatrix}^T$
1. Rotate body frame by $\psi$ about z-axis
1. Rotate body frame by $\theta$ about new y'-axis
1. Rotate body frame by $\phi$ about new x''-axis

*Note:* Names of reference frames that are inter changable:

- body = local
- navigation = fixed = global = inertial

This sequence can be represented by the following:

$$
R_{321} = R_1(\phi) R_2(\theta) R_3(\psi) = R^b_G
$$

Notice the subscript 321, that indicates the order z, y, and then x. Now multiplying those matricies together gives:

$$
R(x)R(y)R(z) = R_{321} = 
\left[\begin{matrix}\cos{\left(y \right)} \cos{\left(z \right)} & \sin{\left(z \right)} \cos{\left(y \right)} & - \sin{\left(y \right)}\\\sin{\left(x \right)} \sin{\left(y \right)} \cos{\left(z \right)} - \sin{\left(z \right)} \cos{\left(x \right)} & \sin{\left(x \right)} \sin{\left(y \right)} \sin{\left(z \right)} + \cos{\left(x \right)} \cos{\left(z \right)} & \sin{\left(x \right)} \cos{\left(y \right)}\\\sin{\left(x \right)} \sin{\left(z \right)} + \sin{\left(y \right)} \cos{\left(x \right)} \cos{\left(z \right)} & - \sin{\left(x \right)} \cos{\left(z \right)} + \sin{\left(y \right)} \sin{\left(z \right)} \cos{\left(x \right)} & \cos{\left(x \right)} \cos{\left(y \right)}\end{matrix}\right]
$$

Using this matrix now allows us to transform a vector in the fixed inertial frame to the rotating body frame.

$$
\begin{bmatrix}x & y & z\end{bmatrix}^b = R_{321} \begin{bmatrix}X & Y & Z\end{bmatrix}^n
$$

- Generally:
    - Fixed to body: $x^b = R(\phi)R(\theta)R(\psi) x^n = R_{321} x^n$
    - Body to fixed: $x^n = R_{321}^{-1} x^b = R_{321}^{T} x^b$

## 1.2 Recovering the Euler Angles

- Conversion from rotation matrix (e.g., $R_{321}$) to Euler is:
    - $R = R_{321}^T = R^G_b$
    - Roll: $\phi = atan2(R_{32}, R_{33})$
    - Pitch: $\theta = -asin(R_{31})$, $\theta \neq \pm 90$
    - Yaw: $\psi = atan2(R_{21}, R_{11})$

## 1.3 Inverse Rotations

The inverse is:

$$
v^b = R_{321} v^n = R^b_n v^n \\
v^n = R_{321}^T v^b = R^n_b v^b
$$

Thus, to move back and forth between reference frames, say body to navigation frame, it can be a simple transpose of the rotation matrix. Now, order matters in this process.

$$
R_{321}(\psi,\theta,\phi) \neq R_{123}(\phi,\theta,\psi)^T
$$

The correct way is:

$$
R_{321}(\psi,\theta,\phi) = R_{123}(-\phi,-\theta,-\psi)^T \\
R_{321}^T = R_z(-\psi)R_y(-\theta)R_x(-\phi) = R_{123}(-\phi,-\theta,-\psi)
$$

Again, becareful of notation. I use $R_{zyx}(rot_z,rot_y,rot_x)$ and $R_{xyz}(rot_x,rot_y,rot_z)$ but not all authors use that notation. Both axis and angles are in order of operation from left to right.

### 1.3.1 Deeper Dive into Euler Matrix Inverse

Let's dive a little deeper into the above. Here, let's do a simple derivation of what the inverse means and perform it  a couple different ways.

So we know the following is true from mathematics:

$$
R(a)^{-1} = R(a)^T \\
(AB)^T = B^TA^T
$$

But let's show the details a little more to prove it by moving individual matrices to the other side of the equation. First let's define what $R_{123}$ is:

$$
R_{123} (c,b,a) = R_3(a) R_2(b) R_1(c)
$$

Now let's derive the inverse o f$R_{321}$:

$$
\begin{align*}
                                 A &= R_{321}(a,b,c) B \\
                                 A &= R_{1}(c) R_{2}(b) R_{3}(a) B \\
                      R_{1}(c)^T A &= R_{2}(b) R_{3}(a) B \\
           R_{2}(b)^T R_{1}(c)^T A &= R_{3}(a) B \\
R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T A &= B
\end{align*}
$$

See how the inverse (or transpose) of $R_{321}$ is not equal to $R_{123}$? or we could use the transpose property above ($(AB)^T = B^TA^T$):

$$
\begin{align*}
                                 A &= R_{321}(a,b,c) B \\
                R_{321}(a,b,c)^T A &= B \\
  [R_{1}(c) R_{2}(b) R_{3}(a)]^T A &= B \\
R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T A &= B \\
                  R_{123}(c,b,a) A &= B
\end{align*}
$$

Notice how the inverse of $R_{321}(a,b,c)$ is $R_{123}(c,b,a)$ with the values swapped around.
Thus, from above, we can state:

$$
\begin{align*}
R_{321}(a,b,c)^{-1} &= R_{321}(a,b,c)^T \\
             &= R_{3}(a)^T R_{2}(b)^T R_{1}(c)^T \\
             &= R_{123}(c,b,a)
\end{align*}
$$

So does this disagree with the above section ... no. Notice the $R_{123}$ is composed of $R_{m}(m)^T$ components which is equal to $R_{m}(-m)$ because the frame is inversed, the angle negative in a non-inversed frame. The easiest way to think of this is, you can represent any rotation by an axis/angle pair. If I invert the rotation, all I have done is flipped the axis and rotated by a positive angel **or** I have left the axis alone and rotated by a negative angle.

# 2.0 Quaternions

Quaternions are another way to represent rotations but with an extra degree of freedom. 

$$
\mathbb{R^4} \Leftrightarrow \mathbb{R^3}
$$

Basically, a rigid body can be rotated angle $\mu$ about an arbitrary moving/fixed axis ($\hat e$) in space. This axis and angle rotation can be represented by a quaternion. Quaternions are composed of a real component (w) and an imaginary component (x,y,z) which has 3 elements. Thus quaternions use 4 variables to represent 3 rotational elements or 3 degrees of freedom (think Euler: roll, pitch, and yaw). This redundant variable allows them to avoid issues seen in Euler angles. 

The only problem with them is you have to be able to visualize 3D rotations in 4D space ... good luck! Thus, for humans to visualize them, we typically transform them back to Euler angles. However, the issues we see in Euler angles (gimbal lock) can reappear when we transform them. Note, the gimbal lock only is an issue in Euler angles, quaternions are fine, it is just the transformation to Euler that can have an issue.

## 2.1 Definition

An issue with quaternions, is not all authors agree on how to write quaternions. Two common conventions are:

|                  | Hamilton             | JPL                  |
|------------------|:---------------------|:---------------------|
| Order            | $[q_w, q_{xyz}]$     | $[q_{xyz}, q_{w}]$   |
| Algebra          | ij=k (right handed)  | ij=k (left handed)   |
| Default Rotation | From local to global | From global to local |
| Notation         | $q=q^G_L$            | $q=q^L_G$            |

- **Hamilton:** $x^G = q \otimes x^L \otimes q^*$
- **JPL:** $x^L = q \otimes x^G \otimes q^*$

For the remainder of this work, we will use the **Hamilton** definition of quaternions.

$$
q = \begin{bmatrix} w & x & y & z \end{bmatrix}  = \begin{bmatrix} q_0 & q_1 & q_2 & q_3 \end{bmatrix} \\
q_{x,y,z} = \hat e \sin( \frac{\mu}{2} ) \\
q_w = \cos(\frac{\mu}{2} )
$$

## 2.2 Basic Mathematics

There are a lot of mathematical things you can do with quaternions, but I won't cover all of that here. I will just cover the most common operations needed for the work I do (robotics, satellites, navigation, etc). 

Quaternion multiplication ($\otimes$) is:

$$
q \otimes p =
\begin{bmatrix}
    q_w & -q_x & -q_y & -q_z \\
    q_x &  q_w & -q_z &  q_y \\
    q_y &  q_z &  q_w & -q_x \\
    q_z & -q_y &  q_x &  q_w
\end{bmatrix} \cdot p = Q(q) \cdot p \\
$$

Now, this is not the only way to write quaternion multiplication. You can also rearrange this:

$$
q \otimes p = P(p)q \\
P(p) =
\begin{bmatrix}
    p_w & -p_x & -p_y & -p_z \\
    p_x &  p_w &  p_z & -p_y \\
    p_y & -p_z &  p_w &  p_x \\
    p_z &  p_y & -p_x &  p_w
\end{bmatrix}
$$

Quaternion differential equation using angular velocity ($\omega$) and quaternion multiplication is:

$$
\dot q = \frac{1}{2} q \otimes w \\
w = \begin{bmatrix} 0 & \omega_b \end{bmatrix}^T
$$

Or use the standard matrix and vector multiplication representation of quaternion multiplication shown above to rewrite this as:

$$
\dot q = \frac{1}{2} W q \\
W =
\begin{bmatrix}
    0   & -w_x & -w_y & -w_z \\
    w_x &    0 &  w_z & -w_y \\
    w_y & -w_z &    0 &  w_x \\
    w_z &  w_y & -w_x &    0
\end{bmatrix}
$$

For discrete timeframes:

$$
q_{k+1} = exp(\frac{1}{2} w \Delta T) \otimes q_k \\
q_{k+1} = q_k + (\frac{1}{2} w \Delta T) \otimes q_k 
$$

## 2.3 Simple Quaternions

Generally I don't need all of the capabilities (or complexity) of quaternion math libraries. Basically I just need a way to convert between Euler and Quaternion representations and have a nice way to print them out.

[squaterion](https://pypi.org/project/squaternion/) is a quaterion library. You can install it with:

```
pip3 install squaternion
```

In [3]:
# create a simple quaterion with no rotation
q = Quaternion()
print(f"Default quaternion: {q}\n")

# My quaternions are immutable so they allways return
# a new quaternion without changing the original
q = Quaternion(1,2,3,4)
print(f"Non-normalized quaternion: {q}")
print(f"Return a normalize quaternion:\n  {q.normalize}")
print(f"Conjugate: {q.conjugate}")

Default quaternion: Quaternion(w=1.0, x=0.0, y=0.0, z=0.0)

Non-normalized quaternion: Quaternion(w=1, x=2, y=3, z=4)
Return a normalize quaternion:
  Quaternion(w=0.18257418583505536, x=0.3651483716701107, y=0.5477225575051661, z=0.7302967433402214)
Conjugate: Quaternion(w=1, x=-2, y=-3, z=-4)


## 2.4 Quaternion to Euler

In [4]:
q = Quaternion.from_euler(30,-60,90,degrees=True)
print(q.to_euler(degrees=True))

(30.000000000000004, -60.00000000000001, 90.00000000000003)


In [5]:
# r = set_r(-45,34,90)
# print("R321 = ")
# r

In [7]:
# remember, R321 and q go in opposite directions, so we
# need to take the conjugate (transpose equivelent) of
# our quaternion to create the matrix
q = Quaternion.from_euler(-45,34,90, degrees=True)
print(f"Euler: {q.to_euler(degrees=True)} deg")
qq = q.conjugate
# print("R321 = ")
# Matrix(qq.to_rot())

Euler: (-44.999999999999986, 34.0, 89.99999999999999) deg


# Setup Symbolics

In [13]:
# for symbolic mathematics
import sympy
from sympy import symbols, sin, cos, pi, simplify
from sympy import latex
from sympy import Matrix
from sympy.algebras.quaternion import Quaternion as sQuaternion

---

# Appendix A: Derive 321 Quaternion

https://www.autonomousrobotslab.com/frame-rotations-and-representations.html

Here we are going to use the `sympy` toolbox and its `Quaternion` (renamed `sQuaternion` to deconflict) and prove some things.

- `sQuaternion` is composed of [a,b,c,d] instead of [w,x,y,z]

In [9]:
r,p,y = symbols("r p y")

In [11]:
def qmult(q,r):
    # performs quaternion multiplication
    # eqn 13
    # [a b c d] = [w x y z]
    w = q.a*r.a - q.b*r.b - q.c*r.c - q.d*r.d
    x = q.b*r.a + q.a*r.b - q.d*r.c + q.c*r.d
    y = q.c*r.a + q.d*r.b + q.a*r.c - q.b*r.d
    z = q.d*r.a - q.c*r.b + q.b*r.c + q.a*r.d
    return sQuaternion(w,x,y,z)

def from_euler(roll, pitch, yaw, degrees=False):
    # creates a quaternion from Euler angles
    if degrees:
        deg2rad = np.pi/180
        roll  *= deg2rad
        pitch *= deg2rad
        yaw   *= deg2rad
    cy = cos(yaw * 0.5)
    sy = sin(yaw * 0.5)
    cr = cos(roll * 0.5)
    sr = sin(roll * 0.5)
    cp = cos(pitch * 0.5)
    sp = sin(pitch * 0.5)

    w = cy * cr * cp + sy * sr * sp
    x = cy * sr * cp - sy * cr * sp
    y = cy * cr * sp + sy * sr * cp
    z = sy * cr * cp - cy * sr * sp

    return sQuaternion(w, x, y, z)

def qclose(a,b):
    def to_tuple(a):
        return (float(a.a), float(a.b), float(a.c), float(a.d))
    aa = to_tuple(a)
    bb = to_tuple(b)
    ans = np.allclose(aa,bb)
    if ans:
        print(f"{Fore.GREEN}{ans}{Fore.RESET}")
    else:
        print(f"{Fore.RED}{ans}{Fore.RESET}")

## Quaternion Representation of R321

Now, remember, rotation matrix $R_{321}$ transforms a vector from global frame to body fixed frame. Since we are using Hamilton quaternions, they perform the *opposite* transfrom, from body fixed to global frame. Thus we really don't want ZXY order, but rather XYZ order ... however, we will show both and the inverse. 

$$
R_{321} = R_x R_y R_z = q_{yaw} \otimes q_{pitch} \otimes q_{roll} \\
q_{321} = q_{yaw} \otimes q_{pitch} \otimes q_{roll} \\
q_{321} = q_{roll}^* \otimes q_{pitch}^* \otimes q_{yaw}^* 
$$

Now, you can do the inverse like this:

$$
q_{123} = q_{roll} \otimes q_{pitch} \otimes q_{yaw} \\
q_{321} = q_{123}^*
$$

Remember to insert **negative roll, pitch, and yaw** for the above $q_{123}^*$.

In [14]:
q = sQuaternion(1,0,0,0); print(f"Basic: {q}")
q = sQuaternion(1,2,3,4); print(f"Another basic: {q}")
print(f"Conjugate: {q.conjugate()}")
print(f"Conjugate: {q.normalize().evalf()}")
print(f"Conjugate: {sympy.N(q.normalize())}")

Basic: 1 + 0*i + 0*j + 0*k
Another basic: 1 + 2*i + 3*j + 4*k
Conjugate: 1 + (-2)*i + (-3)*j + (-4)*k
Conjugate: sqrt(30)/30 + sqrt(30)/15*i + sqrt(30)/10*j + 2*sqrt(30)/15*k
Conjugate: sqrt(30)/30 + sqrt(30)/15*i + sqrt(30)/10*j + 2*sqrt(30)/15*k


In [15]:
# individual quaternion rotations for roll, pitch, and yaw:
qr = from_euler(r,0,0); print(qr)
qp = from_euler(0,p,0); print(qp)
qy = from_euler(0,0,y); print(qy)

cos(0.5*r) + sin(0.5*r)*i + 0*j + 0*k
cos(0.5*p) + 0*i + sin(0.5*p)*j + 0*k
cos(0.5*y) + 0*i + 0*j + sin(0.5*y)*k


In [16]:
# first representation above: q321 = qy x qp x qr
q321 = simplify(qy.mul(qp.mul(qr)))
q321

(sin(0.5*p)*sin(0.5*r)*sin(0.5*y) + cos(0.5*p)*cos(0.5*r)*cos(0.5*y)) + (-sin(0.5*p)*sin(0.5*y)*cos(0.5*r) + sin(0.5*r)*cos(0.5*p)*cos(0.5*y))*i + (sin(0.5*p)*cos(0.5*r)*cos(0.5*y) + sin(0.5*r)*sin(0.5*y)*cos(0.5*p))*j + (-sin(0.5*p)*sin(0.5*r)*cos(0.5*y) + sin(0.5*y)*cos(0.5*p)*cos(0.5*r))*k

So we end up with:

$$
q_r = \cos{\left(0.5 r \right)} + \sin{\left(0.5 r \right)} i + 0 j + 0 k \\
q_p = \cos{\left(0.5 p \right)} + 0 i + \sin{\left(0.5 p \right)} j + 0 k \\
q_y = \cos{\left(0.5 y \right)} + 0 i + 0 j + \sin{\left(0.5 y \right)} k
$$

Or shown another way:

$$
q_r = \begin{bmatrix} \cos(0.5 r) & \sin(0.5r) & 0 & 0 \end{bmatrix} \\
q_p = \begin{bmatrix} \cos(0.5 p) & 0 & \sin(0.5p) & 0 \end{bmatrix} \\
q_y = \begin{bmatrix} \cos(0.5 y) & 0 & 0 & \sin(0.5y) \end{bmatrix} \\
$$

Now multiplying them together:

$$
q_{321} = q_y \otimes q_p \otimes q_r \\
q_{321} = q_r^* \otimes q_p^* \otimes q_y^* \\
q_{321} = \left(\sin{\left(0.5 p \right)} \sin{\left(0.5 r \right)} \sin{\left(0.5 y \right)} + \cos{\left(0.5 p \right)} \cos{\left(0.5 r \right)} \cos{\left(0.5 y \right)}\right) + \left(- \sin{\left(0.5 p \right)} \sin{\left(0.5 y \right)} \cos{\left(0.5 r \right)} + \sin{\left(0.5 r \right)} \cos{\left(0.5 p \right)} \cos{\left(0.5 y \right)}\right) i + \left(\sin{\left(0.5 p \right)} \cos{\left(0.5 r \right)} \cos{\left(0.5 y \right)} + \sin{\left(0.5 r \right)} \sin{\left(0.5 y \right)} \cos{\left(0.5 p \right)}\right) j + \left(- \sin{\left(0.5 p \right)} \sin{\left(0.5 r \right)} \cos{\left(0.5 y \right)} + \sin{\left(0.5 y \right)} \cos{\left(0.5 p \right)} \cos{\left(0.5 r \right)}\right) k
$$

## Inverse $q_{321}$

Like with Euler, order matters.

In [17]:
# here is the inverse order of 321
q123=qmult(qr, qmult(qp, qy))
q123

(-sin(0.5*p)*sin(0.5*r)*sin(0.5*y) + cos(0.5*p)*cos(0.5*r)*cos(0.5*y)) + (sin(0.5*p)*sin(0.5*y)*cos(0.5*r) + sin(0.5*r)*cos(0.5*p)*cos(0.5*y))*i + (sin(0.5*p)*cos(0.5*r)*cos(0.5*y) - sin(0.5*r)*sin(0.5*y)*cos(0.5*p))*j + (sin(0.5*p)*sin(0.5*r)*cos(0.5*y) + sin(0.5*y)*cos(0.5*p)*cos(0.5*r))*k

In [18]:
# REFERENCE
# Make the R321 representation via a quaternion
qa = from_euler(10,20,30,True); print(qa)

0.951548524643789 + 0.0381345764748501*i + 0.189307857412*j + 0.23929833774473*k


In [19]:
# WORKS
# create a quaternion from squaternion, convert it into a sympy
# sQuaternion so we can compare to the above quaternion
sq = Quaternion.from_euler(10,20,30,True); print(sq)
sqq = sQuaternion(*sq) 
qclose(qa, sqq)

Quaternion(w=0.9515485246437886, x=0.03813457647485015, y=0.189307857412, z=0.2392983377447303)
True


In [20]:
# WORKS
# q321 == from_euler
qaa = q321.subs(r,10*np.pi/180).subs(p,20*np.pi/180).subs(y,30*np.pi/180)
print(qaa)
qclose(qa, qaa)

0.951548524643788 + 0.0381345764748501*i + 0.189307857412*j + 0.23929833774473*k
True


In [21]:
# FAIL
# q321 != q123*
qaa = q123.subs(r,10*np.pi/180).subs(p,20*np.pi/180).subs(y,30*np.pi/180).conjugate()
print(qaa)
qclose(qa, qaa)

0.943714364147489 + (-0.127679440695781)*i + (-0.144878125417369)*j + (-0.268535822751569)*k
False


In [22]:
# WORKS
# Just like we showed above with R321 and R123, we had to
# use the opposite (negative) angles
qaa = q123.subs(r,-10*np.pi/180).subs(p,-20*np.pi/180).subs(y,-30*np.pi/180).conjugate()
print(qaa)
qclose(qa, qaa)

0.951548524643788 + 0.0381345764748501*i + 0.189307857412*j + 0.23929833774473*k
True


### $q_{321}$ Using Conjugates

This will do: $q_{321} = q_{roll}^* \otimes q_{pitch}^* \otimes q_{yaw}^*$

In [23]:
# Let's take the conjugate of each and multiply 
# them together as q321 = qr* x qp* x qy* 
# I like this better since it feels more like 
# R321 with the z-axis rotation first
qrr=qr.conjugate()
qpp=qp.conjugate()
qyy=qy.conjugate()
q321alt=simplify(qrr.mul(qpp.mul(qyy)))
print(q321alt)

(sin(0.5*p)*sin(0.5*r)*sin(0.5*y) + cos(0.5*p)*cos(0.5*r)*cos(0.5*y)) + (sin(0.5*p)*sin(0.5*y)*cos(0.5*r) - sin(0.5*r)*cos(0.5*p)*cos(0.5*y))*i + (-sin(0.5*p)*cos(0.5*r)*cos(0.5*y) - sin(0.5*r)*sin(0.5*y)*cos(0.5*p))*j + (sin(0.5*p)*sin(0.5*r)*cos(0.5*y) - sin(0.5*y)*cos(0.5*p)*cos(0.5*r))*k


In [24]:
qx = q321alt.subs(r,10*np.pi/180).subs(p,20*np.pi/180).subs(y,30*np.pi/180).conjugate()
print(qx)
qclose(qa,qx) # compare to from euler

0.951548524643788 + 0.0381345764748501*i + 0.189307857412*j + 0.23929833774473*k
True


In [25]:
print(latex(q321))

\left(\sin{\left(0.5 p \right)} \sin{\left(0.5 r \right)} \sin{\left(0.5 y \right)} + \cos{\left(0.5 p \right)} \cos{\left(0.5 r \right)} \cos{\left(0.5 y \right)}\right) + \left(- \sin{\left(0.5 p \right)} \sin{\left(0.5 y \right)} \cos{\left(0.5 r \right)} + \sin{\left(0.5 r \right)} \cos{\left(0.5 p \right)} \cos{\left(0.5 y \right)}\right) i + \left(\sin{\left(0.5 p \right)} \cos{\left(0.5 r \right)} \cos{\left(0.5 y \right)} + \sin{\left(0.5 r \right)} \sin{\left(0.5 y \right)} \cos{\left(0.5 p \right)}\right) j + \left(- \sin{\left(0.5 p \right)} \sin{\left(0.5 r \right)} \cos{\left(0.5 y \right)} + \sin{\left(0.5 y \right)} \cos{\left(0.5 p \right)} \cos{\left(0.5 r \right)}\right) k


# Test

![](https://upload.wikimedia.org/wikipedia/en/thumb/3/30/Plane_with_ENU_embedded_axes.svg/425px-Plane_with_ENU_embedded_axes.svg.png)

So let's do a simple test of iterating over a bunch of Euler angles, convert them to a quaternion and then back to Euler and see if we get the same answer.

    euler => quaternion => euler

Now, Euler angles have a sinularity around the following locations:

- Roll:  [-$\pi$, $\pi$]
- Pitch: [-$\pi$/2, $\pi$/2]
- Yaw:   [-$\pi$, $\pi$]

Anything outside of these Euler angle ranges will not work, unless you take special percausions. For my applications, this isn't an issue.

So let's run through a range of valid Euler angles and do the transforms and see if we have an issue. Note, because of small rounding errors, we check if the answers are the same within 0.001 degrees. If **no errors print out** and all you see is **Done**, then everything went well.

In [26]:
# run though and make sure there are no errors
# https://en.wikipedia.org/wiki/Euler_angles#Conventions_2
# valid ranges:
# asin: [-pi/2, pi/2]
# cos: [0, pi]
# atan2: [-pi,pi]
#---------------------
# valid euler angles, meaning no gimbal lock issues
# roll: [-pi,pi]
# pitch: [-pi/2, pi/2]
# yaw: [-pi,pi]
delta = 10
for i in range(-179, 180, delta):
    for j in range(-89, 90, delta):
        for k in range(-179,180, delta):
            q = Quaternion.from_euler(i,j,k, degrees=True)  # euler => quat
            e = q.to_euler(degrees=True)     # quat => euler
            for a, b in zip((i,j,k,), e):
                if abs(a - b) > 0.001:  # are the answers within 0.001 degrees?
                    print('-'*40)
                    print('Error')
                    print(i,j,k, '==', e)
                    print(q)
print("Done")

Done


# Appendix B: Euler Angles

### Sympy

Let's use the symbolic toolbox to derive the above equations. 

$$
x = \phi \\
y = \theta \\
z = \psi
$$

In [27]:
x,y,z = symbols("x y z")

In [28]:
# rotate about z-axis
Rz = np.array([
    [cos(z), sin(z), 0],
    [-sin(z), cos(z),0],
    [0,0,1]
])

# rotate about y-axis
Ry = np.array([
    [cos(y), 0, -sin(y)],
    [0,1,0],
    [sin(y),0,cos(y)]
])

# rotate about the x-axis
Rx = np.array([
    [1,0,0],
    [0,cos(x),sin(x)],
    [0, -sin(x),cos(x)]
])

In [29]:
# Helper functions
def set_r(a,b,c):
    d2r = np.pi/180
    r = R321.subs(x,a*d2r).subs(y,b*d2r).subs(z,c*d2r)
    return r

def to_euler(R):
    r2d = 180/np.pi
    R = R.T # why? R is n to b
    r = atan2(R[2,1], R[2,2]) * r2d
    p = asin(-R[2,0]) * r2d
    y = atan2(R[1,0],R[0,0]) * r2d
    return r,p,y

## Deriving the R321 Matrix

In [30]:
R321 = Matrix(Rx @ Ry @ Rz)
R321

Matrix([
[                       cos(y)*cos(z),                         sin(z)*cos(y),       -sin(y)],
[sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(y)*sin(z) + cos(x)*cos(z), sin(x)*cos(y)],
[sin(x)*sin(z) + sin(y)*cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x), cos(x)*cos(y)]])

In [31]:
# if all angles are 0, then this should be the identify matrix
R321.subs(x,0).subs(y,0).subs(z,0)

Matrix([
[1, 0, 0],
[0, 1, 0],
[0, 0, 1]])

In [32]:
r = set_r(-45,23,120)
to_euler(r)

(-45.0, 23.0, 119.99999999999999)

In [33]:
r = set_r(20,-5,45)
to_euler(r)

(19.999999999999996, -5.0, 45.0)

## Testing Order

Let's play around with the order of the matricies and see if they are equal. I will show the **same** thing basically over and over, but doing something different.

In [58]:
# Order maters
# R321 != inv(R123)
R123 = Matrix(Rz @ Ry @ Rx)
R123

Matrix([
[ cos(y)*cos(z),  sin(x)*sin(y)*cos(z) + sin(z)*cos(x), sin(x)*sin(z) - sin(y)*cos(x)*cos(z)],
[-sin(z)*cos(y), -sin(x)*sin(y)*sin(z) + cos(x)*cos(z), sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[        sin(y),                        -sin(x)*cos(y),                        cos(x)*cos(y)]])

In [59]:
invR123 = simplify(R123.inv())
invR123

Matrix([
[                       cos(y)*cos(z),                        -sin(z)*cos(y),         sin(y)],
[sin(x)*sin(y)*cos(z) + sin(z)*cos(x), -sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(y)],
[sin(x)*sin(z) - sin(y)*cos(x)*cos(z),  sin(x)*cos(z) + sin(y)*sin(z)*cos(x),  cos(x)*cos(y)]])

In [60]:
R321 == invR123 # they are not the same

False

In [62]:
# earlier we talked about having to substitute in the opposite angle
# in order to get the right answer
rrneg=R123.subs(x,-x).subs(y,-y).subs(z,-z)
rrneg

Matrix([
[cos(y)*cos(z), sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(z) + sin(y)*cos(x)*cos(z)],
[sin(z)*cos(y), sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[      -sin(y),                        sin(x)*cos(y),                         cos(x)*cos(y)]])

In [63]:
R321 == rrneg.T

True

In [64]:
# done another way
rrneg == simplify(R321.inv())

True

In [65]:
# done another way
# R321 == RxRyRz
R321 == simplify(Matrix(Rx @ Ry @ Rz))

True

In [66]:
# inv(R321) == inv(RxRyRz)
simplify(R321.inv()) == simplify(Matrix(Rx @ Ry @ Rz).inv())

True

In [67]:
# inv(R321) == Rx^T Ry^T Rz^T
simplify(R321.inv()) == simplify(Matrix(Rz.T @ Ry.T @ Rx.T))

True

In [68]:
# this does NOT work ... because everything is opposite
# inv(R321) != RzRyRx
simplify(R321.inv()) == simplify(Matrix(Rz @ Ry @ Rx))

False

In [69]:
# This DOES work because we reversed the angles
# inv(R321) == RzRyRx
simplify(R321.inv()) == simplify(Matrix(Rz @ Ry @ Rx).subs(x,-x).subs(y,-y).subs(z,-z))

True

In [53]:
simplify(Matrix(Rz.T @ Ry.T @ Rx.T))

Matrix([
[cos(y)*cos(z), sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(z) + sin(y)*cos(x)*cos(z)],
[sin(z)*cos(y), sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[      -sin(y),                        sin(x)*cos(y),                         cos(x)*cos(y)]])

In [55]:
simplify(R321.inv())

Matrix([
[cos(y)*cos(z), sin(x)*sin(y)*cos(z) - sin(z)*cos(x),  sin(x)*sin(z) + sin(y)*cos(x)*cos(z)],
[sin(z)*cos(y), sin(x)*sin(y)*sin(z) + cos(x)*cos(z), -sin(x)*cos(z) + sin(y)*sin(z)*cos(x)],
[      -sin(y),                        sin(x)*cos(y),                         cos(x)*cos(y)]])

# Appendix C: Definitions

- **Tait–Bryan angles:** represent rotations about three distinct axes (e.g. x-y-z, or x-y′-z″). It is the convention normally used for aerospace applications, so that zero degrees elevation represents the horizontal attitude. Tait–Bryan angles represent the orientation of the aircraft with respect to the world frame.
    - z-y′-x″ (intrinsic rotations) or x-y-z (extrinsic rotations): the intrinsic rotations are known as: yaw, pitch and roll
- **Euler angles:** proper Euler angles use the same axis for both the first and third elemental rotations (e.g., z-x-z, or z-x′-z″)
- **Intrinsic rotations:** are elemental rotations that occur about the axes of a coordinate system attached to a moving body. Therefore, they change their orientation after each elemental rotation. The body system rotates, while a global system is fixed. 
- **Extrinsic rotations:** are elemental rotations that occur about the axes of the global  coordinate system. The global system rotates, while body is fixed. [correct?]